In [2]:
from openpyxl import Workbook
from openpyxl.styles import Font
from openpyxl.styles.borders import Border, Side
from openpyxl.styles import PatternFill, Alignment
from datetime import date
import random

# Die folgenden imports stammen aus der Datei ../00_common/common.py

import sys
sys.path.append("../00_common")
import common

def get_data():
    d_population = {}
    with open("Population_EU.csv", "r", encoding='utf-8-sig') as file:
        firstline = True
        for line in file:
            line = line.strip()
            line = line.split(";")
            if firstline:
                # grab second value as initial year
                initial_year = int(line[1])
                firstline = False
            else:
                d_country = {}
                country = line[0]
                year = initial_year
                for popul in line[1:]:
                    d_country[year] = int(popul)
                    year += 1
                d_population[country] = d_country  

    d_GDP = {}
    with open("GDP_EU.csv", "r", encoding='utf-8-sig') as file:
        firstline = True
        for line in file:
            line = line.strip()
            line = line.split(";")
            if firstline:
                # grab second value as initial year
                initial_year = int(line[1])
                firstline = False
            else:
                d_country = {}
                country = line[0]
                year = initial_year
                for popul in line[1:]:
                    d_country[year] = int(popul)
                    year += 1
                d_GDP[country] = d_country
          
    d_StockMarket = {}
    with open("StockMarket_Historical_Data.csv", "r", encoding='utf-8-sig') as file:
        firstline = True
        for line in file:
            line = line.strip()
            line = line.split(";")
            if firstline:
                l_company = []
                for company in line[1:]:
                    d_course = {}
                    d_StockMarket[company] = d_course
                    l_company.append(company)
                firstline = False
            else:
                date = line[0]
                for count, value in enumerate(line[1:]):
                    d_StockMarket[l_company[count]][date] = value

    l_names = []
    with open("names_2000.csv", "r", encoding='utf-8-sig') as file:
        for line in file:
            line = line.strip()
            fname, name = line.split(";")
            l_names.append((fname, name))

    # define colors
    r = random.choice(["00", "80", "FF"])
    g = random.choice(["00", "80", "FF"])
    b = random.choice(["00", "80", "FF"])
    color1 = r + g + b 
    if color1 == "FFFFFF" or color1 == "000000":
        color1 = "FF0000"
    r = random.choice(["00", "80", "FF"])
    g = random.choice(["00", "80", "FF"])
    b = random.choice(["00", "80", "FF"])
    color2 = r + g + b
    if color2 == "FFFFFF" or color2 == "000000":
        color2 = "00FF00"
    colors = (color1, color2)            
            
    return d_population, d_GDP, d_StockMarket, l_names, colors

def make_worksheet_1(wb, matr):
    wb.create_sheet("1) Turnover")
    ws = wb["1) Turnover"]
    global l_names
    names_5 = random.sample(l_names, 5)

    ws.append(["Turnover Current Ywear"])
    ws.append(["", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec", "Turnover Year"])
    for name in names_5:
        turnover = []
        for i in range(11):
            turnover.append(random.randint(50000, 150000))
        ws.append([name[0] + " " + name[1]] + turnover)

    for col in "ABCDEFGHIJKLMN":
         ws.column_dimensions[col].width = 15   

    # Hash and hide the matrikelnumber
    seed = "Computer"
    val = hash(str(matr) + seed) % 10000000000
    ws["BB3"].value = val
    ws["BB3"].font = Font(color="FFFFFF")
    return wb

def make_worksheet_2(wb):
    wb.create_sheet("2) Stock Market")
    ws = wb["2) Stock Market"]
    global d_StockMarket
    global colors
    
    l_companies = ["Apple", "Amazon", "BMW", "Ford", "Google", "IBM", "Mercedes", "Meta", "Microsoft", "Oracle", "SAP"]
    comp1, comp2 = random.sample(l_companies, 2)
    comp1_stock = d_StockMarket[comp1]
    comp2_stock = d_StockMarket[comp2]
    ws.append(["Stock Market", "", "", "", "", "", "", comp1, "", "", comp2])
    period = random.choice(["30-Days", "60-Days", "90-Days"])
    ws.append(["Date", comp1, "", "Date", comp2, "", "Date", "Share Price", period, "Normalized", "Share Price", period, "Normalized"])
    ref_year = random.choice(list(range(2014, 2024)))
    day_num = day_num = (date(ref_year, 1, 1) - date(1900, 1, 1)).days + 2 - int(period[:2])
    row_num = 3
    ws.cell(row = 3, column = 7).value = day_num
    ws.cell(row = 3, column = 7).number_format = "DD.MM.YYYY"
    
    for dat, value in comp1_stock.items():
        # Calculate the Excel-Number for the date
        year = 2000 + int(dat[6:])
        month = int(dat[3:5])
        day = int(dat[:2])
        day_num = (date(year, month, day) - date(1900, 1, 1)).days + 2
        ws.cell(row = row_num, column = 1).value = day_num
        value = value.replace(",", ".")
        if value.replace(".", "").isnumeric():
            value = float(value)
        ws.cell(row = row_num, column = 2).value = value
        ws.cell(row = row_num, column = 1).number_format = "DD.MM.YYYY"
        ws.cell(row = row_num, column = 2).number_format = '0.00" €"'
        row_num += 1

    row_num = 3
    for dat, value in comp2_stock.items():
        # Calculate the Excel-Number for the date
        year = 2000 + int(dat[6:])
        month = int(dat[3:5])
        day = int(dat[:2])
        day_num = (date(year, month, day) - date(1900, 1, 1)).days + 2
        ws.cell(row = row_num, column = 4).value = day_num
        value = value.replace(",", ".")
        if value.replace(".", "").isnumeric():
            value = float(value)
        ws.cell(row = row_num, column = 5).value = value
        ws.cell(row = row_num, column = 4).number_format = "DD.MM.YYYY"
        ws.cell(row = row_num, column = 5).number_format = '0.00" €"'
        row_num += 1

    for col in "ADG":
         ws.column_dimensions[col].width = 15

    ws.cell(row = 1, column = 9).fill = PatternFill(start_color=colors[0], end_color=colors[0], fill_type="solid")
    ws.cell(row = 1, column = 9).value = colors[0]
    ws.cell(row = 1, column = 12).fill = PatternFill(start_color=colors[1], end_color=colors[1], fill_type="solid")
    ws.cell(row = 1, column = 12).value = colors[1]

    return wb

def make_worksheet_3(wb):
    wb.create_sheet("3) GDP")
    ws = wb["3) GDP"]
    global d_GDP
    global d_population
    global colors
    
    countries = [["Iceland", "Malta", "Luxembourg", "Cyprus"], ["Estonia", "Slovenia", "Latvia", "Ireland", "Lithuania"], \
                 ["Norway", "Finland", "Denmark", "Sweden"], ["Switzerland", "Austria", "Belgium", "Netherlands"], \
                 ["Croatia", "Slovakia", "Bulgaria", "Czechia", "Hungary"], ["Spain", "Italy", "France", "Germany"], \
                 ["Greece", "Portugal", "Romania", "Poland"]]
    country1, country2 = random.sample(random.choice(countries), 2)
    GDP_country1 = d_GDP[country1]
    GDP_country2 = d_GDP[country2]
    pop_country1 = d_population[country1]
    pop_country2 = d_population[country2]

    ws.append(["GDP and Population"])
    ws.append(["", country1, "", country2])
    ws.append(["Year", "GDP", "Population", "GDP", "Population"])

    row_num = 4
    for year in range(1995, 2024):
        ws.cell(row = row_num, column = 1).value = year
        ws.cell(row = row_num, column = 2).value = GDP_country1[year]
        ws.cell(row = row_num, column = 3).value = pop_country1[year]
        ws.cell(row = row_num, column = 4).value = GDP_country2[year]
        ws.cell(row = row_num, column = 5).value = pop_country2[year]
        row_num += 1

    ws.cell(row = 1, column = 7).fill = PatternFill(start_color=colors[0], end_color=colors[0], fill_type="solid")
    ws.cell(row = 1, column = 7).value = colors[0]
    ws.cell(row = 1, column = 8).fill = PatternFill(start_color=colors[1], end_color=colors[1], fill_type="solid")
    ws.cell(row = 1, column = 8).value = colors[1]
              
    return wb

def make_worksheet_4(wb):
    global colors
    
    wb.create_sheet("4) Scattered-Diagram")
    ws = wb["4) Scattered-Diagram"]
    ws.append(["Size and Weight"])
    ws.append(["Size (cm)", "Weight (kg)", "Sex"])
    num_pers = random.randint(150, 200)
    row_num = 3
    for i in range(num_pers):
        bmi = random.randint(1800, 2800) / 100
        sex = random.choice(["m", "f"])
        if sex == "m":
            size = random.randint(170, 200) / 100
        else:
            size = random.randint(160, 190) / 100
        weight = bmi * size**2
        ws.append([size * 100, weight, sex])
        ws.cell(row = row_num, column = 2).number_format = "0.0"
        row_num += 1

    ws.cell(row = 1, column = 4).fill = PatternFill(start_color=colors[0], end_color=colors[0], fill_type="solid")
    ws.cell(row = 1, column = 4).value = colors[0]
    ws.cell(row = 1, column = 5).fill = PatternFill(start_color=colors[1], end_color=colors[1], fill_type="solid")
    ws.cell(row = 1, column = 5).value = colors[1]

    return wb

def make_workbook(filename, matr):
    wb = Workbook()
    wb = make_worksheet_1(wb, matr)
    wb = make_worksheet_2(wb) 
    wb = make_worksheet_3(wb) 
    wb = make_worksheet_4(wb)
    del wb["Sheet"]
    return wb

# Main Program
l_matr = common.get_Matrikelnummern("Matrikelnummern.csv")
d_population, d_GDP, d_StockMarket, l_names, colors = get_data()
common.create_work_books_in_Excel_Files(l_matr, make_workbook)
#common.create_work_books_in_Abgaben(l_matr, make_workbook)





Go for it
Excel-Files created


In [5]:
import os
os.chdir(".")
print(os.getcwd())

/Users/jacobs/GIT/PC_Praktikum/10_Excel_EN


In [4]:
seed = "abc"
i = str(3207510) + seed 
h = hash(i)
print(h)


1706249285716807316


In [ ]:
values = [234, 456, 567, 789]
for count, value in enumerate(values[1:]):
    print(count, value)

In [ ]:
d0 = date(1900, 1, 1)
d1 = date(2013, 1, 1)
delta = d1 - d0
print(delta.days)

In [ ]:
import datetime

mdate = "2010-10-05"
rdate = "2010-10-05"
mdate1 = datetime.datetime.strptime(mdate, "%Y-%m-%d").date()
rdate1 = datetime.datetime.strptime(rdate, "%Y-%m-%d").date()
delta =  (mdate1 - rdate1).days
print(
print(delta +2)

In [ ]:
c = "Belgium,Bulgaria,Czechia,Denmark,Germany,Estonia,Ireland,Greece,Spain,France,Croatia,Italy,Cyprus,Latvia,Lithuania,Luxembourg,Hungary,Netherlands,Austria,Poland,Portugal,Romania,Slovenia,Slovakia,Finland,Sweden,Norway,Switzerland"
c = c.split(",")
print(c)